In [1]:
import os
import re
from pathlib import Path
from typing import List, Dict
import logging
from bs4 import BeautifulSoup, NavigableString

from config.settings import settings

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
def extract_sections(soup: BeautifulSoup, uri: str) -> List[Dict]:
        sections = soup.find_all("section")
        section_list = []
        for i, section in enumerate(sections):
            section_id = section.get("id")
            section_text = extract_text_from_section(section)
            if section_id:
                section_data = {
                    "source": f"{uri}#{section_id}",
                    "text": section_text,
                    "previous_section": section_list[i-1]['source'] if i > 0 else None,
                    "next_section": None,
                    "metadata": {
                        "page_heading": soup.find("h1").get_text().strip() if soup.find("h1") else Path(uri).stem,
                        "section_id": section_id
                    }
                }
                if i > 0:
                    section_list[i-1]['next_section'] = section_data['source']
                section_list.append(section_data)
        return section_list

In [4]:
def extract_text_from_section(section) -> str:
        texts = []
        for element in section.children:
            if isinstance(element, NavigableString):
                if element.strip():
                    texts.append(element.strip())
            elif element.name != 'section':
                texts.append(element.get_text().strip())
        return clean_text(" ".join(texts))

In [5]:
def path_to_uri(path: Path, scheme: str = "https://", domain: str = "docs.fastht.ml") -> str:
        relative_path = str(path.relative_to(settings.RAW_DATA_DIR)).replace("\\", "/")
        return scheme + domain + "/" + relative_path


def clean_text(text: str) -> str:
    # Replace multiple newlines with a single space
    return re.sub(r'\s+', ' ', text).strip()


def process_html_files(html_files_path: List[Path]) -> List[Dict]:
    docs_text = []
    for record in html_files_path:
        print("processing: ", record)
        with open(record, "r", encoding="utf-8") as html_file:
            soup = BeautifulSoup(html_file, "html.parser")
        uri = path_to_uri(path=record)
        sections = extract_sections(soup, uri)
        docs_text.append(sections)
    return docs_text

In [6]:
html_files_path = [path for path in settings.RAW_DATA_DIR.rglob("*.html") if not path.is_dir()]
docs_text = process_html_files(html_files_path)
print(f"Total documents processed: {len(docs_text)}")

processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/index.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/ref/defining_xt_component.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/ref/live_reload.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/ref/handlers.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/api/oauth.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/api/core.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/api/cli.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/api/fastapp.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/api/components.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/data/raw-data/api/xtend.html
processing:  /home/amrit/data-projects/llms/fasthtml-docs-bot/dat

In [7]:
docs_text[0:1]

[[{'source': 'https://docs.fastht.ml/index.html#installation',
   'text': 'Installation Since fasthtml is a Python library, you can install it with: pip install python-fasthtml In the near future, we hope to add component libraries that can likewise be installed via pip .',
   'previous_section': None,
   'next_section': 'https://docs.fastht.ml/index.html#usage',
   'metadata': {'page_heading': 'FastHTML', 'section_id': 'installation'}},
  {'source': 'https://docs.fastht.ml/index.html#usage',
   'text': 'Usage For a minimal app, create a file “main.py” as follows: main.py from fasthtml.common import * app,rt = fast_app() @rt(\'/\') def get(): return Div(P(\'Hello World!\'), hx_get="/change") serve() Running the app with python main.py prints out a link to your running app: http://localhost:5001 . Visit that link in your browser and you should see a page with the text “Hello World!”. Congratulations, you’ve just created your first FastHTML app! Adding interactivity is surprisingly easy,

In [3]:
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
class Preprocessor:
    def __init__(self, chunk_size: int, chunk_overlap: int):
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", " ", ""],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len
        )

    def chunk_section(self, section: Dict[str, str]) -> List[Dict[str, str]]:
        chunks = self.text_splitter.create_documents(
            texts=[section["text"]],
            metadatas=[{
                "source": section["source"],
                "previous_section": section.get("previous_section"),
                "next_section": section.get("next_section"),
                "metadata": section.get("metadata")
            }]
        )
        return [{
            "text": chunk.page_content,
            "source": chunk.metadata["source"],
            "previous_section": chunk.metadata["previous_section"],
            "next_section": chunk.metadata["next_section"],
            "metadata": chunk.metadata["metadata"]
        } for chunk in chunks]

In [10]:
if __name__ == "__main__":

    preprocessor = Preprocessor(chunk_size=settings.CHUNK_SIZE, chunk_overlap=settings.CHUNK_OVERLAP)
    chunked_docs_text = []
    for doc in docs_text:
        for section in doc:
            chunked_sections = preprocessor.chunk_section(section)
            chunked_docs_text.extend(chunked_sections)
    
    print(f"Total chunks created: {len(chunked_docs_text)}")

Total chunks created: 534


In [11]:
chunked_docs_text[0:5]

[{'text': 'Installation Since fasthtml is a Python library, you can install it with: pip install python-fasthtml In the near future, we hope to add component libraries that can likewise be installed via pip .',
  'source': 'https://docs.fastht.ml/index.html#installation',
  'previous_section': None,
  'next_section': 'https://docs.fastht.ml/index.html#usage',
  'metadata': {'page_heading': 'FastHTML', 'section_id': 'installation'}},
 {'text': 'Usage For a minimal app, create a file “main.py” as follows: main.py from fasthtml.common import * app,rt = fast_app() @rt(\'/\') def get(): return Div(P(\'Hello World!\'), hx_get="/change") serve() Running the app with python main.py prints out a link to your running app: http://localhost:5001 . Visit that link in your browser and you should see a page with the text “Hello World!”. Congratulations, you’ve just created your first FastHTML app! Adding interactivity is surprisingly easy, thanks to',
  'source': 'https://docs.fastht.ml/index.html#us

In [62]:
from typing import List, Dict
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [63]:
# get_embedding_model: Returns an embedding model based on the specified model name.
# OpenAIEmbeddings: Uses OpenAI’s API for embedding.
# HuggingFaceEmbeddings: Uses Hugging Face’s models for embedding.

def get_embedding_model(embedding_model_name, model_kwargs, encode_kwargs):
    if embedding_model_name == "text-embedding-ada-002":
        embedding_model = OpenAIEmbeddings(
            model = embedding_model_name,
            openai_api_base = os.environ["OPENAI_API_BASE"],
            openai_api_key = os.environ["OPENAI_API_KEY"]
        )
    else:
        embedding_model = HuggingFaceEmbeddings(
            model_name = embedding_model_name, # also works with model_path
            model_kwargs = model_kwargs,
            encode_kwargs = encode_kwargs
        )
    return embedding_model

# EmbedChunks: A class to embed chunks using the specified model.
# init: Initializes the embedding model.
# call: Embeds the text in the batch and returns the embeddings along with the original text and source.

class EmbedChunks:
    def __init__(self, model_name):
        self.embedding_model = get_embedding_model(
            embedding_model_name = model_name,
            model_kwargs = {"device": "cuda"},
            encode_kwargs = {"device": "cuda", "batch_size": 100}
        )

    def __call__(self, batch):
        embeddings = self.embedding_model.embed_documents(batch["text"])
        return {"text": batch["text"], "source": batch["source"], "embeddings": embeddings}


class Embedder:
    def __init__(self, model_name: str):
        self.embedding_model = get_embedding_model(
            model_name=model_name,
            model_kwargs={"device": "cuda"},
            encode_kwargs={"device": "cuda", "batch_size": 100}
        )

    def embed_chunks(self, chunks: List[Dict[str, str]]) -> List[Dict[str, str]]:
        texts = [chunk["text"] for chunk in chunks]
        embeddings = self.embedding_model.embed_documents(texts)
        return [
            {"text": chunk["text"], "source": chunk["source"], "embedding": embedding}
            for chunk, embedding in zip(chunks, embeddings)
        ]

In [14]:
if __name__ == "__main__":
    embedder = Embedder(model_name=settings.EMBEDDING_MODEL_NAME)
    embedded_chunks = embedder.embed_chunks(chunked_docs_text)

    print(f"Total embeddings created: {len(embedded_chunks)}")

/home/amrit/miniforge3/envs/llmdev/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
INFO:datasets:PyTorch version 2.4.1 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-base


Total embeddings created: 534


In [64]:
# src/storage/vector_store.py
import os
from langchain_chroma import Chroma
from langchain_core.documents import Document

In [65]:
class VectorStore:
    def __init__(self, embedding_model, persist_directory: Path):
        self.vector_store = Chroma(
            collection_name="fasthtml_docs_db",
            embedding_function=embedding_model,
            persist_directory=os.path.abspath(persist_directory)
        )

    def add_documents(self, documents: List[Dict[str, str]]):
        docs = [
            Document(page_content=doc["text"], metadata={"source": doc["source"]})
            for doc in documents
        ]
        self.vector_store.add_documents(docs)
        logger.info(f"Added {len(docs)} documents to the vector store")

    def similarity_search(self, query: str, k: int = 5) -> List[Dict[str, str]]:
        results = self.vector_store.similarity_search_with_score(query, k=k)
        return [
            {
                "text": doc.page_content,
                "source": doc.metadata.get("source", ""),
                "score": score
            }
            for doc, score in results
        ]

In [73]:
if __name__ == "__main__":
    # Initialize the embedding model
    embedding_model = HuggingFaceEmbeddings(
        model_name=settings.EMBEDDING_MODEL_NAME,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"device": "cuda", "batch_size": 100}
    )

    # Initialize the vector store
    vectorStore = VectorStore(embedding_model=embedding_model, persist_directory=settings.VECTOR_STORE_DIR)
    
    # Assuming embedded_chunks is already defined and contains the embedded documents
    vectorStore.add_documents(embedded_chunks)

    # Assuming you have a client instance to interact with the Chroma DB
    client = vectorStore.vector_store._client
    collection = client.get_collection(name="fasthtml_docs_db")

    print(f"Total items in {collection}: {collection.count()}")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-base
INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


Total items in name='fasthtml_docs_db' id=UUID('f5836f62-38aa-40d4-add4-b2b4d063ece3') metadata=None: 534


In [12]:
vectorStore.similarity_search('how to install fasthtml?')

[{'text': 'Installing FastHTML FastHTML is just Python . Installation is often done with pip: pip install python-fasthtml',
  'source': 'https://docs.fastht.ml/tutorials/tutorial_for_web_devs.html#installing-fasthtml',
  'score': 0.1384994089603424},
 {'text': 'Install FastHTML For Mac, Windows and Linux, enter: pip install python-fasthtml',
  'source': 'https://docs.fastht.ml/tutorials/e2e.html#install-fasthtml',
  'score': 0.1573334038257599},
 {'text': 'Installation pip install python-fasthtml',
  'source': 'https://docs.fastht.ml/tutorials/quickstart_for_web_devs.html#installation',
  'score': 0.1606634110212326},
 {'text': 'Installation Since fasthtml is a Python library, you can install it with: pip install python-fasthtml In the near future, we hope to add component libraries that can likewise be installed via pip .',
  'source': 'https://docs.fastht.ml/index.html#installation',
  'score': 0.1731264591217041},
 {'text': 'FastHTML Basics FastHTML is just Python . You can install 

### Generate Response

In [66]:
import openai
import time

from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_huggingface import HuggingFaceEndpoint

In [67]:
def response_stream(chat_completion, llm):
    if llm.startswith("gpt"):
        for chunk in chat_completion:
            content = chunk.choices[0].delta.content
            if content is not None:
                yield content
    else:
        for chunk in chat_completion:
            yield chunk

def prepare_response(chat_completion, stream, llm):
    if stream:
        return response_stream(chat_completion, llm)
    else:
        if llm.startswith("gpt"):
            return chat_completion.choices[0].message.content
        else:
            return chat_completion


def get_client(llm):
    if llm.startswith("gpt"):
        base_url = os.environ["OPENAI_API_BASE"]
        api_key = os.environ["OPENAI_API_KEY"]
        client = openai.OpenAI(base_url=base_url, api_key=api_key)
    else: 
        #base_url = os.environ["HUGGINGFACEHUB_API_BASE"]
        api_key = os.environ["HUGGINGFACEHUB_API_TOKEN"]
        client = HuggingFaceEndpoint(huggingfacehub_api_token=api_key, repo_id=llm)
    
    return client

In [68]:
def generate_response(
    llm, temperature=0.0, stream=True, 
    system_content="", assistant_content="", user_content="", 
    max_retries=1, retry_interval=60):
    """Generate response from an LLM"""
    retry_count = 0
    client = get_client(llm=llm)
    
    prompt = [("system", system_content), ("assistant", assistant_content), ("user", user_content)]
    messages = [{"role": role, "content": content} for role, content in prompt if content]

    while retry_count <= max_retries:
        try:
            if llm.startswith("gpt"):
                chat_completion = client.chat.completions.create(
                    model=llm,
                    temperature=temperature,
                    stream=stream,
                    messages=messages,
                )
            else:
                chat_completion = client.invoke(
                    repo_id=llm,
                    temperature=temperature,
                    streaming=stream,
                    input=messages,
                )
            return prepare_response(chat_completion, stream=stream, llm=llm)

        except Exception as e:
            print(f'Exception: {e}')
            time.sleep(retry_interval) # default is pre-minute rate limits
            retry_count += 1
    return ""    

In [53]:
context_results = vectorStore.similarity_search('how to install fasthtml?')
context = [item["text"] for item in context_results]
print(context)

['Installing FastHTML FastHTML is just Python . Installation is often done with pip: pip install python-fasthtml', 'Install FastHTML For Mac, Windows and Linux, enter: pip install python-fasthtml', 'Installation pip install python-fasthtml', 'Installation Since fasthtml is a Python library, you can install it with: pip install python-fasthtml In the near future, we hope to add component libraries that can likewise be installed via pip .', 'FastHTML Basics FastHTML is just Python . You can install it with pip install python-fasthtml . Extensions/components built for it can likewise be distributed via PyPI or as simple Python files. The core usage of FastHTML is to define routes, and then to define what to do at each route. This is similar to the FastAPI web framework (in fact we implemented much of the functionality to match the FastAPI usage examples), but where FastAPI focuses on returning JSON data to build APIs, FastHTML']


In [54]:
# Generate response
query = 'how to install fasthtml?'
response = generate_response(
    llm="mistralai/Mistral-7B-Instruct-v0.3",
    temperature=0.1,
    stream=True,
    system_content="Answer the query using the context provided. Be succinct.",
    user_content=f"query: {query}, context: {context}"
)

# Stream response
for content in response:
    print(content, end='', flush=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/amrit/.cache/huggingface/token
Login successful


To install FastHTML, use the command `pip install python-fasthtml`. This command works on Mac, Windows, and Linux systems. FastHTML is a Python library, so it can be installed using pip.

In [55]:
import tiktoken


def get_num_tokens(text):
    enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))


def trim(text, max_context_length):
    enc = tiktoken.get_encoding("cl100k_base")
    return enc.decode(enc.encode(text)[:max_context_length])

In [79]:
class QueryAgent:
    def __init__(self, embedding_model_name="thenlper/gte-base",
                 llm="mistralai/Mistral-7B-Instruct-v0.3", temperature=0.1, 
                 max_context_length=4096, system_content="", assistant_content=""
                ):
        # Embedding model
        self.embedding_model = get_embedding_model(
            embedding_model_name = embedding_model_name,
            model_kwargs = {"device": "cuda"},
            encode_kwargs = {"device": "cuda", "batch_size": 100}
        )

        # Context length (restrict input length to 50% of total context length)
        max_context_length = int(0.5*max_context_length)

        # LLM
        self.llm = llm
        self.temperature = temperature
        self.context_length = max_context_length - get_num_tokens(system_content + assistant_content)
        self.system_content = system_content
        self.assistant_content = assistant_content

    def __call__(self, query, num_chunks=5, stream=True):
        # Get sources and context
        # Initialize the vector store
        vectorStore = VectorStore(embedding_model=embedding_model_name, persist_directory=settings.VECTOR_STORE_DIR)
        context_results = vectorStore.similarity_search(query=query)

        # Generate response
        context = [item["text"] for item in context_results]
        sources = [item["source"] for item in context_results]
        user_content = f"query: {query}, context: {context}"
        answer = generate_response(
            llm=self.llm,
            temperature=self.temperature,
            stream=stream,
            system_content=self.system_content,
            assistant_content=self.assistant_content,
            user_content=trim(user_content, self.context_length))

        # Result
        result = {
            "question": query,
            "sources": sources,
            "answer": answer,
            "llm": self.llm,
        }
        return result

# context_results = vectorStore.similarity_search('how to install fasthtml?')
# context = [item["text"] for item in context_results]

# query = 'how to install fasthtml?'
# response = generate_response(
#     llm="mistralai/Mistral-7B-Instruct-v0.3",
#     temperature=0.1,
#     stream=True,
#     system_content="Answer the query using the context provided. Be succinct.",
#     user_content=f"query: {query}, context: {context}"
# )

In [80]:
import json
from config.config import MAX_CONTEXT_LENGTHS

In [81]:
embedding_model_name = "thenlper/gte-base"
llm = "mistralai/Mistral-7B-Instruct-v0.3"

In [82]:
query = 'how to install fasthtml?'
system_content = "Answer the query using the context provided. Be succinct."
agent = QueryAgent(
    embedding_model_name=embedding_model_name,
    llm=llm,
    max_context_length=MAX_CONTEXT_LENGTHS[llm],
    system_content=system_content)
result = agent(query=query, stream=False)
print(json.dumps(result, indent=2))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-base


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/amrit/.cache/huggingface/token
Login successful
{
  "question": "how to install fasthtml?",
  "sources": [
    "https://docs.fastht.ml/tutorials/tutorial_for_web_devs.html#installing-fasthtml",
    "https://docs.fastht.ml/tutorials/e2e.html#install-fasthtml",
    "https://docs.fastht.ml/tutorials/quickstart_for_web_devs.html#installation",
    "https://docs.fastht.ml/index.html#installation",
    "https://docs.fastht.ml/tutorials/by_example.html#fasthtml-basics"
  ],
  "answer": "\n\nTo install FastHTML, use the command `pip install python-fasthtml`. This command works on Mac, Windows, and Linux systems. FastHTML is a Python library, so it can be installed using pip.",
  "llm": "mistralai/Mist

I am building a Q&A bot for Fasthtml docs website so that users can ask any query related to Fasthtml. Complete FastHTML Docs Bot Project Structure and Code is included. I have included current code and structure. I have made some changes to code so, i have added new code as well. your task to include the changes to the code base. if possible, try include these best pratices in the code further optimize this for production environment.
1. Use pydantic for configuration management and data validation.
2. Consider using asyncio for asynchronous processing where applicable.
3. Implement logging instead of print statements for better debugging.
4. Use tqdm for progress bars in long-running processes.
5. Consider using typer for creating command-line interfaces for your scripts.
6. For large-scale processing, consider using Ray for distributed processing

In [ ]:
# current code and Project Structure

fasthtml-docs-bot/
│
├── src/
│   ├── data/
│   │   ├── __init__.py
│   │   ├── downloader.py
│   │   ├── extractor.py
│   │   └── preprocessor.py
│   │
│   ├── embedding/
│   │   ├── __init__.py
│   │   └── embedder.py
│   │
│   ├── storage/
│   │   ├── __init__.py
│   │   └── vector_store.py
│   │
│   ├── query/
│   │   ├── __init__.py
│   │   └── searcher.py
│   │
│   └── utils/
│       ├── __init__.py
│       └── helpers.py
│
├── scripts/
│   ├── download_docs.py
│   ├── process_docs.py
│   ├── create_embeddings.py
│   └── query_docs.py
│
├── tests/
│   ├── test_downloader.py
│   ├── test_extractor.py
│   ├── test_embedder.py
│   └── test_searcher.py
│
├── data/
│   ├── raw-data/
│   ├── processed-data/
│   └── docs_db/
│
├── config/
│   ├── __init__.py
│   └── settings.py
│   └── config.py
│
├── requirements.txt
├── setup.py
├── README.md
└── .gitignore

# Code for each file

# config/settings.py

from pydantic import BaseSettings

class Settings(BaseSettings):
    BASE_URL: str = "https://docs.fastht.ml/"
    RAW_DATA_DIR: str = "data/raw-data"
    PROCESSED_DATA_DIR: str = "data/processed-data"
    VECTOR_STORE_DIR: str = "data/fasthtml_docs_db"
    CHUNK_SIZE: int = 500
    CHUNK_OVERLAP: int = 50
    EMBEDDING_MODEL_NAME: str = "thenlper/gte-base"

    class Config:
        env_file = ".env"

settings = Settings()

# src/data/downloader.py

import os
from pathlib import Path
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm import tqdm
import logging

from config.settings import settings

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class HTMLDownloader:
    def __init__(self, base_url: str, output_dir: str):
        self.base_url = base_url
        self.output_dir = Path(output_dir)
        self.downloaded_urls = set()
        self.output_dir.mkdir(parents=True, exist_ok=True)

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
    def fetch_url(self, url: str) -> requests.Response:
        response = requests.get(url)
        response.raise_for_status()
        return response

    def download_page(self, url: str) -> list:
        if url in self.downloaded_urls:
            return []

        try:
            response = self.fetch_url(url)
        except requests.exceptions.RequestException as e:
            logger.error(f'Failed to download: {url} due to {e}')
            return []

        self.downloaded_urls.add(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        file_name = url.split('/')[-1] or 'index.html'
        file_path = self.output_dir / file_name
        
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(soup.prettify())
        logger.info(f'Downloaded: {url}')

        return [
            urljoin(url, link['href'])
            for link in soup.find_all('a', href=True)
            if urljoin(url, link['href']).startswith(self.base_url) and 
               urljoin(url, link['href']).endswith('.html')
        ]

    def download_all(self):
        to_download = [self.base_url]
        with ThreadPoolExecutor(max_workers=10) as executor:
            with tqdm(total=len(to_download), desc="Downloading pages") as pbar:
                while to_download:
                    futures = [executor.submit(self.download_page, url) for url in to_download]
                    to_download = []
                    for future in as_completed(futures):
                        result = future.result()
                        if result:
                            to_download.extend(result)
                        pbar.update(1)
                    pbar.total = pbar.n + len(to_download)

# src/data/extractor.py

from pathlib import Path
from bs4 import BeautifulSoup
from typing import List, Dict
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class HTMLExtractor:
    @staticmethod
    def extract_sections(html_file: Path) -> List[Dict[str, str]]:
        with open(html_file, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

        sections = soup.find_all("section")
        section_list = []
        for section in sections:
            section_id = section.get("id")
            section_text = HTMLExtractor.extract_text_from_section(section)
            if section_id:
                uri = HTMLExtractor.path_to_uri(path=html_file)
                section_list.append({"source": f"{uri}#{section_id}", "text": section_text})
        
        logger.info(f"Extracted {len(section_list)} sections from {html_file}")
        return section_list

    @staticmethod
    def extract_text_from_section(section) -> str:
        texts = []
        for elem in section.children:
            if isinstance(elem, NavigableString):
                if elem.strip():
                    texts.append(elem.strip())
            elif elem.name == 'section':
                continue
            else:
                texts.append(elem.get_text().strip())
        return "\n".join(texts)

    @staticmethod
    def path_to_uri(path: Path, scheme: str = "https://", domain: str = "docs.fastht.ml") -> str:
        return scheme + domain + str(path).split("raw-data")[-1]

# src/data/preprocessor.py

from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter

class Preprocessor:
    def __init__(self, chunk_size: int, chunk_overlap: int):
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", " ", ""],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len
        )

    def chunk_section(self, section: Dict[str, str]) -> List[Dict[str, str]]:
        chunks = self.text_splitter.create_documents(
            texts=[section["text"]],
            metadatas=[{"source": section["source"]}]
        )
        return [{"text": chunk.page_content, "source": chunk.metadata["source"]} for chunk in chunks]

# src/embedding/embedder.py

from typing import List, Dict
from langchain_huggingface import HuggingFaceEmbeddings

class Embedder:
    def __init__(self, model_name: str):
        self.embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": "cuda"},
            encode_kwargs={"device": "cuda", "batch_size": 100}
        )

    def embed_chunks(self, chunks: List[Dict[str, str]]) -> List[Dict[str, str]]:
        texts = [chunk["text"] for chunk in chunks]
        embeddings = self.embedding_model.embed_documents(texts)
        return [
            {"text": chunk["text"], "source": chunk["source"], "embedding": embedding}
            for chunk, embedding in zip(chunks, embeddings)
        ]

# src/storage/vector_store.py

from langchain_chroma import Chroma
from langchain_core.documents import Document
from typing import List, Dict
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class VectorStore:
    def __init__(self, embedding_model, persist_directory: str):
        self.vector_store = Chroma(
            collection_name="fasthtml_docs",
            embedding_function=embedding_model,
            persist_directory=persist_directory
        )

    def add_documents(self, documents: List[Dict[str, str]]):
        docs = [
            Document(page_content=doc["text"], metadata={"source": doc["source"]})
            for doc in documents
        ]
        self.vector_store.add_documents(docs)
        logger.info(f"Added {len(docs)} documents to the vector store")

    def similarity_search(self, query: str, k: int = 5) -> List[Dict[str, str]]:
        results = self.vector_store.similarity_search_with_score(query, k=k)
        return [
            {
                "text": doc.page_content,
                "source": doc.metadata.get("source", ""),
                "score": score
            }
            for doc, score in results
        ]

# src/query/searcher.py

from typing import List, Dict
from src.storage.vector_store import VectorStore

class Searcher:
    def __init__(self, vector_store: VectorStore):
        self.vector_store = vector_store

    def search(self, query: str, k: int = 5) -> List[Dict[str, str]]:
        return self.vector_store.similarity_search(query, k)

# src/utils/helpers.py

import psutil
import GPUtil

def print_resource_usage():
    cpu_percent = psutil.cpu_percent()
    mem_percent = psutil.virtual_memory().percent
    gpus = GPUtil.getGPUs()
    gpu_util = gpus[0].load * 100 if gpus else 0
    print(f"CPU: {cpu_percent}%, Memory: {mem_percent}%, GPU: {gpu_util}%")

# scripts/download_docs.py

from src.data.downloader import HTMLDownloader
from config.settings import settings

def main():
    downloader = HTMLDownloader(settings.BASE_URL, settings.RAW_DATA_DIR)
    downloader.download_all()

if __name__ == "__main__":
    main()

# scripts/process_docs.py

from pathlib import Path
from src.data.extractor import HTMLExtractor
from src.data.preprocessor import Preprocessor
from config.settings import settings

def main():
    extractor = HTMLExtractor()
    preprocessor = Preprocessor(settings.CHUNK_SIZE, settings.CHUNK_OVERLAP)

    raw_data_dir = Path(settings.RAW_DATA_DIR)
    processed_data_dir = Path(settings.PROCESSED_DATA_DIR)
    processed_data_dir.mkdir(parents=True, exist_ok=True)

    for html_file in raw_data_dir.glob("*.html"):
        sections = extractor.extract_sections(html_file)
        chunks = []
        for section in sections:
            chunks.extend(preprocessor.chunk_section(section))
        
        output_file = processed_data_dir / f"{html_file.stem}.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(chunks, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    main()

# scripts/create_embeddings.py

import json
from pathlib import Path
from src.embedding.embedder import Embedder
from src.storage.vector_store import VectorStore
from config.settings import settings

def main():
    embedder = Embedder(settings.EMBEDDING_MODEL_NAME)
    vector_store = VectorStore(embedder.embedding_model, settings.VECTOR_STORE_DIR)

    processed_data_dir = Path(settings.PROCESSED_DATA_DIR)

    for json_file in processed_data_dir.glob("*.json"):
        with open(json_file, "r", encoding="utf-8") as f:
            chunks = json.load(f)
        
        embedded_chunks = embedder.embed_chunks(chunks)
        vector_store.add_documents(embedded_chunks)

if __name__ == "__main__":
    main()

# scripts/query_docs.py

from src.query.searcher import Searcher
from src.storage.vector_store import VectorStore
from src.embedding.embedder import Embedder
from config.settings import settings

def main():
    embedder = Embedder(settings.EMBEDDING_MODEL_NAME)
    vector_store = VectorStore(embedder.embedding_model, settings.VECTOR_STORE_DIR)
    searcher = Searcher(vector_store)

    while True:
        query = input("Enter your query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        results = searcher.search(query)
        for i, result in enumerate(results, 1):
            print(f"\nResult {i}:")
            print(f"Score: {result['score']}")
            print(f"Source: {result['source']}")
            print(f"Text: {result['text'][:200]}...")  # Display first 200 characters

if __name__ == "__main__":
    main()

# requirements.txt

requests==2.26.0
beautifulsoup4==4.10.0
tenacity==8.0.1
tqdm==4.62.3
pydantic==1.8.2
langchain==0.0.184
langchain-openai==0.0.2
langchain-huggingface==0.0.3
langchain-chroma==0.0.4
psutil==5.8.0
gputil==1.4.0

# setup.py

from setuptools import setup, find_packages

setup(
    name="fasthtml-docs-bot",
    version="0.1",
    packages=find_packages(),
    install_requires=[
        "requests",
        "beautifulsoup4",
        "tenacity",
        "tqdm",
        "pydantic",
        "langchain",
        "langchain-openai",
        "langchain-huggingface",
        "langchain-chroma",
        "psutil",
        "gputil",
    ],
    entry_points={
        "console_scripts": [
            "download-docs=scripts.download_docs:main",
            "process-docs=scripts.process_docs:main",
            "create-embeddings=scripts.create_embeddings:main",
            "query-docs=scripts.query_docs:main",
        ],
    },
)

# README.md

# FastHTML Docs Bot

This project is a documentation bot for FastHTML, capable of downloading HTML documents, processing them, creating embeddings, and enabling semantic search queries.

## Installation

1. Clone the repository:
   ```
   git

In [2]:
I have made few changes to add additional functionalities. Review the files and let me if any optimization is possible. Include the changes to our existing code files and create new files using best practices.
Code:

```
# extract content
import os
import re
from pathlib import Path
from typing import List, Dict
import logging
from bs4 import BeautifulSoup, NavigableString

from config.settings import settings

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def extract_sections(soup: BeautifulSoup, uri: str) -> List[Dict]:
        sections = soup.find_all("section")
        section_list = []
        for i, section in enumerate(sections):
            section_id = section.get("id")
            section_text = extract_text_from_section(section)
            if section_id:
                section_data = {
                    "source": f"{uri}#{section_id}",
                    "text": section_text,
                    "previous_section": section_list[i-1]['source'] if i > 0 else None,
                    "next_section": None,
                    "metadata": {
                        "page_heading": soup.find("h1").get_text().strip() if soup.find("h1") else Path(uri).stem,
                        "section_id": section_id
                    }
                }
                if i > 0:
                    section_list[i-1]['next_section'] = section_data['source']
                section_list.append(section_data)
        return section_list

def extract_text_from_section(section) -> str:
        texts = []
        for element in section.children:
            if isinstance(element, NavigableString):
                if element.strip():
                    texts.append(element.strip())
            elif element.name != 'section':
                texts.append(element.get_text().strip())
        return clean_text(" ".join(texts))

def path_to_uri(path: Path, scheme: str = "https://", domain: str = "docs.fastht.ml") -> str:
        relative_path = str(path.relative_to(settings.RAW_DATA_DIR)).replace("\\", "/")
        return scheme + domain + "/" + relative_path


def clean_text(text: str) -> str:
    # Replace multiple newlines with a single space
    return re.sub(r'\s+', ' ', text).strip()


def process_html_files(html_files_path: List[Path]) -> List[Dict]:
    docs_text = []
    for record in html_files_path:
        print("processing: ", record)
        with open(record, "r", encoding="utf-8") as html_file:
            soup = BeautifulSoup(html_file, "html.parser")
        uri = path_to_uri(path=record)
        sections = extract_sections(soup, uri)
        docs_text.append(sections)
    return docs_text

html_files_path = [path for path in settings.RAW_DATA_DIR.rglob("*.html") if not path.is_dir()]
docs_text = process_html_files(html_files_path)
print(f"Total documents processed: {len(docs_text)}")

# Preprocessing
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter

class Preprocessor:
    def __init__(self, chunk_size: int, chunk_overlap: int):
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", " ", ""],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len
        )

    def chunk_section(self, section: Dict[str, str]) -> List[Dict[str, str]]:
        chunks = self.text_splitter.create_documents(
            texts=[section["text"]],
            metadatas=[{
                "source": section["source"],
                "previous_section": section.get("previous_section"),
                "next_section": section.get("next_section"),
                "metadata": section.get("metadata")
            }]
        )
        return [{
            "text": chunk.page_content,
            "source": chunk.metadata["source"],
            "previous_section": chunk.metadata["previous_section"],
            "next_section": chunk.metadata["next_section"],
            "metadata": chunk.metadata["metadata"]
        } for chunk in chunks]

if __name__ == "__main__":

    preprocessor = Preprocessor(chunk_size=settings.CHUNK_SIZE, chunk_overlap=settings.CHUNK_OVERLAP)
    chunked_docs_text = []
    for doc in docs_text:
        for section in doc:
            chunked_sections = preprocessor.chunk_section(section)
            chunked_docs_text.extend(chunked_sections)
    
    print(f"Total chunks created: {len(chunked_docs_text)}")

# Embedding content
from typing import List, Dict
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# get_embedding_model: Returns an embedding model based on the specified model name.
# OpenAIEmbeddings: Uses OpenAI’s API for embedding.
# HuggingFaceEmbeddings: Uses Hugging Face’s models for embedding.

def get_embedding_model(embedding_model_name, model_kwargs, encode_kwargs):
    if embedding_model_name == "text-embedding-ada-002":
        embedding_model = OpenAIEmbeddings(
            model = embedding_model_name,
            openai_api_base = os.environ["OPENAI_API_BASE"],
            openai_api_key = os.environ["OPENAI_API_KEY"]
        )
    else:
        embedding_model = HuggingFaceEmbeddings(
            model_name = embedding_model_name, # also works with model_path
            model_kwargs = model_kwargs,
            encode_kwargs = encode_kwargs
        )
    return embedding_model

# EmbedChunks: A class to embed chunks using the specified model.
# init: Initializes the embedding model.
# call: Embeds the text in the batch and returns the embeddings along with the original text and source.

class EmbedChunks:
    def __init__(self, model_name):
        self.embedding_model = get_embedding_model(
            embedding_model_name = model_name,
            model_kwargs = {"device": "cuda"},
            encode_kwargs = {"device": "cuda", "batch_size": 100}
        )

    def __call__(self, batch):
        embeddings = self.embedding_model.embed_documents(batch["text"])
        return {"text": batch["text"], "source": batch["source"], "embeddings": embeddings}


class Embedder:
    def __init__(self, model_name: str):
        self.embedding_model = get_embedding_model(
            model_name=model_name,
            model_kwargs={"device": "cuda"},
            encode_kwargs={"device": "cuda", "batch_size": 100}
        )

    def embed_chunks(self, chunks: List[Dict[str, str]]) -> List[Dict[str, str]]:
        texts = [chunk["text"] for chunk in chunks]
        embeddings = self.embedding_model.embed_documents(texts)
        return [
            {"text": chunk["text"], "source": chunk["source"], "embedding": embedding}
            for chunk, embedding in zip(chunks, embeddings)
        ]

if __name__ == "__main__":
    embedder = Embedder(model_name=settings.EMBEDDING_MODEL_NAME)
    embedded_chunks = embedder.embed_chunks(chunked_docs_text)

    print(f"Total embeddings created: {len(embedded_chunks)}")

# store vectors
# src/storage/vector_store.py
import os
from langchain_chroma import Chroma
from langchain_core.documents import Document

class VectorStore:
    def __init__(self, embedding_model, persist_directory: Path):
        self.vector_store = Chroma(
            collection_name="fasthtml_docs_db",
            embedding_function=embedding_model,
            persist_directory=os.path.abspath(persist_directory)
        )

    def add_documents(self, documents: List[Dict[str, str]]):
        docs = [
            Document(page_content=doc["text"], metadata={"source": doc["source"]})
            for doc in documents
        ]
        self.vector_store.add_documents(docs)
        logger.info(f"Added {len(docs)} documents to the vector store")

    def similarity_search(self, query: str, k: int = 5) -> List[Dict[str, str]]:
        results = self.vector_store.similarity_search_with_score(query, k=k)
        return [
            {
                "text": doc.page_content,
                "source": doc.metadata.get("source", ""),
                "score": score
            }
            for doc, score in results
        ]

if __name__ == "__main__":
    # Initialize the embedding model
    embedding_model = HuggingFaceEmbeddings(
        model_name=settings.EMBEDDING_MODEL_NAME,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"device": "cuda", "batch_size": 100}
    )

    # Initialize the vector store
    vectorStore = VectorStore(embedding_model=embedding_model, persist_directory=settings.VECTOR_STORE_DIR)
    
    # Assuming embedded_chunks is already defined and contains the embedded documents
    vectorStore.add_documents(embedded_chunks)

    # Assuming you have a client instance to interact with the Chroma DB
    client = vectorStore.vector_store._client
    collection = client.get_collection(name="fasthtml_docs_db")

    print(f"Total items in {collection}: {collection.count()}")

# Generate Response
import openai
import time

from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_huggingface import HuggingFaceEndpoint

def response_stream(chat_completion, llm):
    if llm.startswith("gpt"):
        for chunk in chat_completion:
            content = chunk.choices[0].delta.content
            if content is not None:
                yield content
    else:
        for chunk in chat_completion:
            yield chunk

def prepare_response(chat_completion, stream, llm):
    if stream:
        return response_stream(chat_completion, llm)
    else:
        if llm.startswith("gpt"):
            return chat_completion.choices[0].message.content
        else:
            return chat_completion


def get_client(llm):
    if llm.startswith("gpt"):
        base_url = os.environ["OPENAI_API_BASE"]
        api_key = os.environ["OPENAI_API_KEY"]
        client = openai.OpenAI(base_url=base_url, api_key=api_key)
    else:
        #base_url = os.environ["HUGGINGFACEHUB_API_BASE"]
        api_key = os.environ["HUGGINGFACEHUB_API_TOKEN"]
        client = HuggingFaceEndpoint(huggingfacehub_api_token=api_key, repo_id=llm)
    
    return client

def generate_response(
    llm, temperature=0.0, stream=True, 
    system_content="", assistant_content="", user_content="", 
    max_retries=1, retry_interval=60):
    """Generate response from an LLM"""
    retry_count = 0
    client = get_client(llm=llm)
    
    prompt = [("system", system_content), ("assistant", assistant_content), ("user", user_content)]
    messages = [{"role": role, "content": content} for role, content in prompt if content]

    while retry_count <= max_retries:
        try:
            if llm.startswith("gpt"):
                chat_completion = client.chat.completions.create(
                    model=llm,
                    temperature=temperature,
                    stream=stream,
                    messages=messages,
                )
            else:
                chat_completion = client.invoke(
                    repo_id=llm,
                    temperature=temperature,
                    streaming=stream,
                    input=messages,
                )
            return prepare_response(chat_completion, stream=stream, llm=llm)

        except Exception as e:
            print(f'Exception: {e}')
            time.sleep(retry_interval) # default is pre-minute rate limits
            retry_count += 1
    return ""    

context_results = vectorStore.similarity_search('how to install fasthtml?')
context = [item["text"] for item in context_results]
print(context)

# Generate response
query = 'how to install fasthtml?'
response = generate_response(
    llm="mistralai/Mistral-7B-Instruct-v0.3",
    temperature=0.1,
    stream=True,
    system_content="Answer the query using the context provided. Be succinct.",
    user_content=f"query: {query}, context: {context}"
)

# Stream response
for content in response:
    print(content, end='', flush=True)

# Create Query agent
import tiktoken


def get_num_tokens(text):
    enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))


def trim(text, max_context_length):
    enc = tiktoken.get_encoding("cl100k_base")
    return enc.decode(enc.encode(text)[:max_context_length])

class QueryAgent:
    def __init__(self, embedding_model_name="thenlper/gte-base",
                 llm="mistralai/Mistral-7B-Instruct-v0.3", temperature=0.1, 
                 max_context_length=4096, system_content="", assistant_content=""
                ):
        # Embedding model
        self.embedding_model = get_embedding_model(
            embedding_model_name = embedding_model_name,
            model_kwargs = {"device": "cuda"},
            encode_kwargs = {"device": "cuda", "batch_size": 100}
        )

        # Context length (restrict input length to 50% of total context length)
        max_context_length = int(0.5*max_context_length)

        # LLM
        self.llm = llm
        self.temperature = temperature
        self.context_length = max_context_length - get_num_tokens(system_content + assistant_content)
        self.system_content = system_content
        self.assistant_content = assistant_content

    def __call__(self, query, num_chunks=5, stream=True):
        # Get sources and context
        # Initialize the vector store
# VectorStore is class defined above for storing vectors. use proper import to access this class
        vectorStore = VectorStore(embedding_model=embedding_model_name, persist_directory=settings.VECTOR_STORE_DIR)
        context_results = vectorStore.similarity_search(query=query)

        # Generate response
        context = [item["text"] for item in context_results]
        sources = [item["source"] for item in context_results]
        user_content = f"query: {query}, context: {context}"
        answer = generate_response(
            llm=self.llm,
            temperature=self.temperature,
            stream=stream,
            system_content=self.system_content,
            assistant_content=self.assistant_content,
            user_content=trim(user_content, self.context_length))

        # Result
        result = {
            "question": query,
            "sources": sources,
            "answer": answer,
            "llm": self.llm,
        }
        return result

import json
from config.config import MAX_CONTEXT_LENGTHS

embedding_model_name = "thenlper/gte-base"
llm = "mistralai/Mistral-7B-Instruct-v0.3"

query = 'how to install fasthtml?'
system_content = "Answer the query using the context provided. Be succinct."
agent = QueryAgent(
    embedding_model_name=embedding_model_name,
    llm=llm,
    max_context_length=MAX_CONTEXT_LENGTHS[llm],
    system_content=system_content)
result = agent(query=query, stream=False)
print(json.dumps(result, indent=2))


# config/config.py

# Embedding dimensions
EMBEDDING_DIMENSIONS = {
    "thenlper/gte-base": 768,
    "thenlper/gte-large": 1024,
    "BAAI/bge-large-en": 1024,
    "text-embedding-ada-002": 1536,
    "gte-large-fine-tuned": 1024,
}

# Maximum context lengths
MAX_CONTEXT_LENGTHS = {
    "gpt-4": 8192,
    "gpt-3.5-turbo": 4096,
    "gpt-3.5-turbo-16k": 16384,
    "gpt-4-1106-preview": 128000,
    "meta-llama/Llama-2-7b-chat-hf": 4096,
    "meta-llama/Llama-2-13b-chat-hf": 4096,
    "meta-llama/Llama-2-70b-chat-hf": 4096,
    "meta-llama/Llama-3-8b-chat-hf": 8192,
    "meta-llama/Llama-3-70b-chat-hf": 8192,
    "codellama/CodeLlama-34b-Instruct-hf": 16384,
    "mistralai/Mistral-7B-Instruct-v0.1": 65536,
    "mistralai/Mixtral-8x7B-Instruct-v0.1": 32768,
    "mistralai/Mixtral-8x22B-Instruct-v0.1": 65536,
    "mistralai/Mistral-7B-Instruct-v0.3": 32768
}

# Pricing per 1M tokens
PRICING = {
    "gpt-3.5-turbo": {"prompt": 1.5, "sampled": 2},
    "gpt-4": {"prompt": 30, "sampled": 60},
    "gpt-4-1106-preview": {"prompt": 10, "sampled": 30},
    "llama-2-7b-chat-hf": {"prompt": 0.15, "sampled": 0.15},
    "llama-2-13b-chat-hf": {"prompt": 0.25, "sampled": 0.25},
    "llama-2-70b-chat-hf": {"prompt": 1, "sampled": 1},
    "llama-3-8b-chat-hf": {"prompt": 0.15, "sampled": 0.15},
    "llama-3-70b-chat-hf": {"prompt": 1, "sampled": 1},
    "codellama-34b-instruct-hf": {"prompt": 1, "sampled": 1},
    "mistral-7b-instruct-v0.1": {"prompt": 0.15, "sampled": 0.15},
    "mixtral-8x7b-instruct-v0.1": {"prompt": 0.50, "sampled": 0.50},
    "mixtral-8x22b-instruct-v0.1": {"prompt": 0.9, "sampled": 0.9},
}


# config/settings.py

from pathlib import Path
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    BASE_DIR: Path = Path(__file__).resolve().parent.parent
    BASE_URL: str = "https://docs.fastht.ml/"
    RAW_DATA_DIR: Path = BASE_DIR / "data" / "raw-data"
    PROCESSED_DATA_DIR: Path = BASE_DIR / "data" / "processed-data"
    VECTOR_STORE_DIR: Path = BASE_DIR / "data" / "docs_db"
    CHUNK_SIZE: int = 500
    CHUNK_OVERLAP: int = 50
    EMBEDDING_MODEL_NAME: str = "thenlper/gte-base"

    class Config:
        env_file = ".env"
        extra = "allow"  # Allow extra fields

settings = Settings()
```

SyntaxError: invalid syntax (2463408557.py, line 1)